In [8]:
#Load necessary libraries
from bioservices.kegg import KEGG
import pandas as pd
import progressbar
k = KEGG()

In [9]:
#Load the conversion table between affymetrix and common gene names
conversion_table = pd.read_csv('datasets/conversion_table.txt', delimiter='\t')


In [ ]:
#Here, we create a concatination of all the pathways for all the genes in the dataset
file = open("pathways.txt", "a")
pathway = {}
bar = progressbar.ProgressBar(maxval=conversion_table['gene_name'].size, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
for gene in conversion_table['gene_name']:
    temp_pathway = k.get_pathway_by_gene(gene, "hsa") # "hsa" assumes that all the genes corresponds to homo sapiens 
    if temp_pathway != None:
        for key in temp_pathway.keys():
            if key not in pathway.keys():
                file.write(str(i) + "\t" + str(key) +"\t"+ str(temp_pathway[key]))
    i = i+1
    bar.update(i)
bar.finish()
file.close() 